# IMPORTACIÓN DE LIBRERÍAS

In [ ]:
import pandas as pd
import numpy as np
import os
import PIL
import seaborn as sns
import pickle
from PIL import *
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.applications import DenseNet121
from keras.models import Model, load_model
from keras.layers import *
from keras.initializers import glorot_uniform
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
#from tensorflow.python.keras import * -> PARA LAS NUEVAS VERSIONES DE TF, HACE QUE NO FUNCIONE
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, optimizers
from keras.applications import ResNet50
from keras import backend as K
#from keras import optimizers -> PARA LAS NUEVAS VERSIONES DE TF, HACE QUE NO FUNCIONE
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from google.colab.patches import cv2_imshow
import copy
from scipy.interpolate import griddata

# CARGA DE DATOS

In [ ]:
data_dir = '/UPC/Tesis/Datasetv4'
label_to_text = {0: 'Pterigión grave', 1: 'Pterigión leve', 2: 'Sin pterigión'}
data = tf.keras.utils.image_dataset_from_directory(data_dir)

In [ ]:
# Convertir data a tipo lista
data_list = list(data.as_numpy_iterator())

# Inicializar listas para almacenar temporalmente los datos
temp_images = []
temp_labels = []

for arr in data_list:
    temp_images.append(arr[0])  # Añadir imagenes a la lista temporal
    temp_labels.append(arr[1])  # Añadir etiquetas a la lista temporal

# Concatenar a lo largo de un nuevo eje para obtener un solo arreglo para imágenes y etiquetas respectivamente
images = np.concatenate(temp_images, axis=0)
labels = np.concatenate(temp_labels, axis=0)

temp_images.clear()
temp_labels.clear()
data_list = []
data = None

### Visualización de dataset

In [ ]:
def graphic_plots(columns, images_set, labels_set):
    _, ax = plt.subplots(ncols=columns, nrows=columns, figsize=(20,20))
    img = images_set[:columns*columns]
    for idx in range(columns):
        for idy in range(columns):
            ax[idx][idy].imshow(img[idx * columns + idy].astype(int))
            label_idx = labels_set[idx * columns + idy]
            ax[idx][idy].title.set_text(label_to_text[label_idx])

In [ ]:
graphic_plots(2, images, labels)

### Estadísticas de dataset

In [ ]:
def plot_label_distribution(_labels):
    # Contar las ocurrencias de cada etiqueta
    unique_labels, counts = np.unique(_labels, return_counts=True)

    label_texts = [label_to_text[l] for l in unique_labels]

    # Crear un gráfico de torta
    plt.figure(figsize=(8, 8))
    plt.pie(counts, labels=label_texts, autopct='%1.1f%%', startangle=140)
    plt.title('Distribución de etiquetas')
    plt.show()

In [ ]:
plot_label_distribution(labels)

# AUMENTO DE DATOS

### Definición de funciones

In [ ]:
def gaussian_filter(sigma, num_channels=3):
    size = int(6*sigma + 1)
    if size % 2 == 0:  # Ensure odd size
        size += 1

    # Create a 1D Gaussian kernel
    x = tf.range(-size // 2 + 1, size // 2 + 1, dtype=tf.float32)
    gauss = tf.exp(-tf.pow(x, 2) / (2 * tf.pow(tf.cast(sigma, tf.float32), 2)))
    gauss /= tf.reduce_sum(gauss)

    # Convert to 2D Gaussian kernel
    gauss_kernel = tf.einsum('i,j->ij', gauss, gauss)
    gauss_kernel = gauss_kernel[:, :, tf.newaxis, tf.newaxis]
    gauss_kernel = tf.tile(gauss_kernel, [1, 1, num_channels, 1])

    return gauss_kernel

In [ ]:
def augment_image(image, delta, contrast_factor, stddev_noise, sigma_blur, color_correction, gamma, sharpen):
    # Ajustar brillo
    image = tf.image.adjust_brightness(image, delta=delta)

    # Ajustar contraste
    image = tf.image.adjust_contrast(image, contrast_factor=contrast_factor)

    # Añadir ruido gaussiano
    noise = tf.random.normal(shape=tf.shape(image), mean=0., stddev=stddev_noise, dtype=tf.float32)
    image = tf.add(image, noise)

    # Aplicar desenfoque gaussiano
    image = tf.nn.depthwise_conv2d(image[tf.newaxis, ...], gaussian_filter(sigma_blur), strides=[1, 1, 1, 1], padding='SAME')[0]

    # Color correction
    image *= color_correction

    # Gamma correction
    image = tf.image.adjust_gamma(image, gamma=gamma)

    # Sharpening
    no_sharpen = tf.constant([[0, 0, 0], [0, 1, 0], [0, 0, 0]], dtype=tf.float32)[:, :, tf.newaxis, tf.newaxis]
    full_sharpen = tf.constant([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]], dtype=tf.float32)[:, :, tf.newaxis, tf.newaxis]
    sharpen_filter = sharpen * full_sharpen + (1 - sharpen) * no_sharpen
    sharpen_filter = tf.tile(sharpen_filter, [1, 1, 3, 1])
    image = tf.nn.depthwise_conv2d(image[tf.newaxis, ...], sharpen_filter, strides=[1, 1, 1, 1], padding='SAME')[0]

    # Asegurarse de que la imagen está en el rango [0, 255]
    image = tf.clip_by_value(image, 0, 255)

    return image

In [ ]:
# 1er Set
modified_images_set_1 = copy.copy(images)
modified_images_set_1 = tf.image.convert_image_dtype(modified_images_set_1, dtype=tf.float32)

# Ajustes personalizados para la ampliación de datos
params = {
    "delta": -10,  # Ejemplo de ajuste de brillo
    "contrast_factor": 0.85,  # Ejemplo de ajuste de contraste
    "stddev_noise": 5,  # Ejemplo de ajuste de ruido
    "sigma_blur": 1.0,  # Ejemplo de ajuste de desenfoque
    "color_correction": (1.0, 1.0, 1.2),  # Ajuste de corrección de color (R, G, B)
    "gamma": 1.05,  # Ajuste de corrección gamma
    "sharpen": 1.1  # Ajuste de agudización
}

modified_images_set_1 = tf.map_fn(lambda img: augment_image(img, **params), modified_images_set_1)
modified_images_set_1 = (modified_images_set_1.numpy())

In [ ]:
# 2do Set
modified_images_set_2 = copy.copy(images)
modified_images_set_2 = tf.image.convert_image_dtype(modified_images_set_2, dtype=tf.float32)

# Ajustes personalizados para la ampliación de datos
params = {
    "delta": 0,  # Ejemplo de ajuste de brillo
    "contrast_factor": 0.8,  # Ejemplo de ajuste de contraste
    "stddev_noise": 5,  # Ejemplo de ajuste de ruido
    "sigma_blur": 1.5,  # Ejemplo de ajuste de desenfoque
    "color_correction": (1.2, 1.0, 0.8),  # Ajuste de corrección de color (R, G, B)
    "gamma": 1.0,  # Ajuste de corrección gamma
    "sharpen": 1.1  # Ajuste de agudización
}

modified_images_set_2 = tf.map_fn(lambda img: augment_image(img, **params), modified_images_set_2)
modified_images_set_2 = (modified_images_set_2.numpy())

In [ ]:
# 3er Set
modified_images_set_3 = copy.copy(images)
modified_images_set_3 = tf.image.convert_image_dtype(modified_images_set_3, dtype=tf.float32)

# Ajustes personalizados para la ampliación de datos
params = {
    "delta": 50,  # Ejemplo de ajuste de brillo
    "contrast_factor": 0.7,  # Ejemplo de ajuste de contraste
    "stddev_noise": 2,  # Ejemplo de ajuste de ruido
    "sigma_blur": 1.0,  # Ejemplo de ajuste de desenfoque
    "color_correction": (0.9, 1.0, 1.0),  # Ajuste de corrección de color (R, G, B)
    "gamma": 0.97,  # Ajuste de corrección gamma
    "sharpen": 1.5  # Ajuste de agudización
}

modified_images_set_3 = tf.map_fn(lambda img: augment_image(img, **params), modified_images_set_3)
modified_images_set_3 = (modified_images_set_3.numpy())

In [ ]:
# CONCATENACIÓN
augmented_images = np.concatenate((images, modified_images_set_1, modified_images_set_2, modified_images_set_3), axis=0)
augmented_labels = np.concatenate((labels, labels, labels, labels), axis=0)
modified_images_set_1 = modified_images_set_2 = modified_images_set_3 = []

In [ ]:
def interpolate_nans(image):
    # Crea una matriz de coordenadas para la imagen
    x, y = np.indices(image.shape)

    # Encuentra los valores NaN en la imagen
    nan_mask = np.isnan(image)

    # Encuentra los valores no-NaN en la imagen
    not_nan_mask = ~nan_mask

    # Obtiene las coordenadas y los valores de los píxeles no-NaN
    coords_not_nan = np.array([x[not_nan_mask], y[not_nan_mask]]).T
    values_not_nan = image[not_nan_mask]

    # Interpola para obtener los valores NaN
    interpolated_values = griddata(coords_not_nan, values_not_nan, (x[nan_mask], y[nan_mask]), method='nearest')

    # Reemplaza los valores NaN en la imagen original
    image[nan_mask] = interpolated_values

    return image

# Para aplicar esto a todas las imágenes en el array 'augmented_images'
for i in range(augmented_images.shape[0]):
    for c in range(augmented_images.shape[3]):
        augmented_images[i, :, :, c] = interpolate_nans(augmented_images[i, :, :, c])


In [ ]:
# GRAFICAR
r = np.random.randint(1, len(images))
n = 4

temp_list = augmented_images[r:len(augmented_images):len(images)]
temp_list_l = augmented_labels[r:len(augmented_labels):len(images)]
fig, ax = plt.subplots(ncols=n, nrows=n, figsize=(20, 20))

for idx in range(n):
    for idy in range(n):
        ax[idy][idx].imshow(augmented_images[r + idy + idx * len(images)].astype(int))
        ax[idy][idx].title.set_text(augmented_labels[r + idy + idx * len(images)])

In [ ]:
images = labels = []

In [ ]:
graphic_plots(2, temp_list, temp_list_l)

In [ ]:
def flip_images_by_label(flip_y, flip_x, label, images_set, labels_set):
    # Extrae las imágenes que coinciden con la etiqueta deseada
    selected_images = images_set[labels_set == label]

    # Convierte las imágenes a tensores y normaliza los valores de los píxeles al rango [0, 1]
    selected_images = tf.convert_to_tensor(selected_images, dtype=tf.float32) / 255.0

    # Aplica los flips según los booleanos flip_x y flip_y
    if flip_y:
        selected_images = tf.image.flip_left_right(selected_images)
    if flip_x:
        selected_images = tf.image.flip_up_down(selected_images)

    # Ajusta el tono y la saturación
    delta_hue = tf.random.uniform(shape=[], minval=-0.05, maxval=0.05)
    factor_saturation = tf.random.uniform(shape=[], minval=1.25, maxval=1.75)
    selected_images = tf.image.adjust_hue(selected_images, delta_hue)
    selected_images = tf.image.adjust_saturation(selected_images, factor_saturation)

    # Convierte los valores de los píxeles de vuelta al rango original [0, 255]
    selected_images = tf.clip_by_value(selected_images * 255.0, 0, 255)

    # Crea un array de etiquetas para las imágenes seleccionadas
    selected_labels = np.full((selected_images.shape[0],), label)

    return selected_images.numpy(), selected_labels

In [ ]:
aug_images_1, aug_labels_1 = flip_images_by_label(True, False, 2, augmented_images, augmented_labels)
aug_images_2, aug_labels_2 = flip_images_by_label(False, True, 2, augmented_images, augmented_labels)
aug_images_3, aug_labels_3 = flip_images_by_label(True, True, 2, augmented_images, augmented_labels)

In [ ]:
aug_images_4, aug_labels_4 = flip_images_by_label(True, False, 1, augmented_images, augmented_labels)

In [ ]:
graphic_plots(4, aug_images_1, aug_labels_1)

In [ ]:
graphic_plots(4, aug_images_2, aug_labels_2)

In [ ]:
graphic_plots(4, aug_images_3, aug_labels_3)

In [ ]:
graphic_plots(4, aug_images_4, aug_labels_4)

In [ ]:
# CONCATENACIÓN
augmented_images = np.concatenate((augmented_images, aug_images_1, aug_images_2, aug_images_3, aug_images_4), axis=0)
augmented_labels = np.concatenate((augmented_labels, aug_labels_1, aug_labels_2, aug_labels_3, aug_labels_4), axis=0)

In [ ]:
aug_images_1 = aug_images_2 = aug_images_3 = aug_images_4 = []
aug_labels_1 = aug_labels_2 = aug_labels_3 = aug_labels_4 = []

In [ ]:
plot_label_distribution(augmented_labels)

# NORMALIZACION DE DATOS

## Normalizar dimensión de imagenes

In [ ]:
# Se divide el valor de cada pixel por 255 para que contengan valores entre 0 y 1
augmented_images = augmented_images/255

In [ ]:
X = augmented_images.copy()

## Normalizar tipos de datos

In [ ]:
# Convertimos el tipo array a float32
X = np.asarray(X).astype(np.float32)
X.shape

In [ ]:
y = np.eye(3)[augmented_labels]
y.shape

In [ ]:
y

In [ ]:
# Dividir el dataframe en conjunto de entrenamiento, test y validación

from sklearn.model_selection import train_test_split

X_train, X_Test, y_train, y_Test = train_test_split(X, y, test_size = 0.2, shuffle = True)
X_val, X_Test, y_val, y_Test = train_test_split(X_Test, y_Test, test_size = 0.5, shuffle = True)

In [ ]:
print(X_val.shape, y_val.shape)

In [ ]:
print(X_Test.shape, y_Test.shape)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
y_Test

In [ ]:
def res_block(_X, filter, stage):

    # Bloque Convolucional
    _X_copy = _X

    f1 , f2, f3 = filter

    # Camino Principal
    _X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_conv_a', kernel_initializer= glorot_uniform(seed = 0))(_X)
    _X = MaxPool2D((2,2))(_X)
    _X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_a')(_X)
    _X = Activation('relu')(_X)

    _X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_conv_b', kernel_initializer= glorot_uniform(seed = 0))(_X)
    _X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_b')(_X)
    _X = Activation('relu')(_X)

    _X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_c', kernel_initializer= glorot_uniform(seed = 0))(_X)
    _X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_c')(_X)


    # Camino Corto
    _X_copy = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_copy', kernel_initializer= glorot_uniform(seed = 0))(_X_copy)
    _X_copy = MaxPool2D((2,2))(_X_copy)
    _X_copy = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_copy')(_X_copy)

    # Añadir
    _X = Add()([_X,_X_copy])
    _X = Activation('relu')(_X)

    # Bloque de Identidad 1
    _X_copy = _X


    # Camino Principal
    _X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_1_a', kernel_initializer= glorot_uniform(seed = 0))(_X)
    _X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_a')(_X)
    _X = Activation('relu')(_X)

    _X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_1_b', kernel_initializer= glorot_uniform(seed = 0))(_X)
    _X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_b')(_X)
    _X = Activation('relu')(_X)

    _X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_1_c', kernel_initializer= glorot_uniform(seed = 0))(_X)
    _X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_c')(_X)

    # Añadir
    _X = Add()([_X,_X_copy])
    _X = Activation('relu')(_X)

    # Bloque de Identidad 2
    _X_copy = _X


    # Camino Principal
    _X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_2_a', kernel_initializer= glorot_uniform(seed = 0))(_X)
    _X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_a')(_X)
    _X = Activation('relu')(_X)

    _X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_2_b', kernel_initializer= glorot_uniform(seed = 0))(_X)
    _X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_b')(_X)
    _X = Activation('relu')(_X)

    _X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_2_c', kernel_initializer= glorot_uniform(seed = 0))(_X)
    _X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_c')(_X)

    # Añadir
    _X = Add()([_X,_X_copy])
    _X = Activation('relu')(_X)

    return _X

In [ ]:
input_shape = (256, 256, 3)

# Tamaño del tensor de entrada
X_input = Input(input_shape)

# Zero-padding
X = ZeroPadding2D((3, 3))(X_input)

# 1 - Fase
X = Conv2D(64, (7, 7), strides= (2, 2), name = 'conv1', kernel_initializer= glorot_uniform(seed = 0))(X)
X = BatchNormalization(axis =3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3, 3), strides= (2, 2))(X)

# 2 - Fase
X = res_block(X, filter= [64, 64, 256], stage= 2)

# 3 - Fase
X = res_block(X, filter= [128, 128, 512], stage= 3)

# 4 - Fase
# X = res_block(X, filter= [256, 256, 1024], stage= 4)

# Average Pooling
X = AveragePooling2D((4, 4), name = 'Averagea_Pooling')(X)

# Capa Final
X = Flatten()(X)
X = Dense(3, activation = 'softmax', name = 'Dense_final', kernel_initializer= glorot_uniform(seed=0))(X)

model_2_emotion = Model(inputs= X_input, outputs = X, name = 'Resnet18')

model_2_emotion.summary()


In [ ]:
# Entrenar la red
model_2_emotion.compile(optimizer = "Adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
# Recordemos que el primer modelo de puntos faciales clave se guardó con: FacialKeyPoints_weights.hdf5 and FacialKeyPoints-model.json

# Usamos la parada temprana para salir del entenamiento si el error de validación
# no decrece después de cierto número de epochs (paciencia)
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 20)

# Guardamos el mejor modelo con menor error de validación
checkpointer = ModelCheckpoint(filepath = "pterygium_detection.hdf5", verbose = 1,
                               save_best_only=True)

In [ ]:
history = model_2_emotion.fit(X_train, y_train,
                              validation_data=(X_val, y_val), steps_per_epoch=len(X_train) // 64,
                              epochs= 20, callbacks=[ checkpointer, earlystopping])

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Guardar la arquitectura del modelo en un JSON para su futuro uso

model_json = model_2_emotion.to_json()
with open("detection-model.json","w") as json_file:
    json_file.write(model_json)

In [ ]:
with open('detection-model.json', 'r') as json_file:
    json_savedModel= json_file.read()

# Cargamos la arquitectura del modelo
model_2_emotion = tf.keras.models.model_from_json(json_savedModel)
model_2_emotion.load_weights('pterygium_detection.hdf5')
model_2_emotion.compile(optimizer = "Adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
score = model_2_emotion.evaluate(X_Test, y_Test)
print('Accuracy en la fase de Test: {}'.format(score[1]))

In [ ]:
history.history.keys()

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
image_test = cv2.imread('/UPC/Tesis/Repositories/pteriscope-detection_model/test_grave7.jpg')
resize = tf.image.resize(image_test, (256, 256))
yhat = model_2_emotion.predict(np.expand_dims(resize/255, 0))
yhat

In [ ]:
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label='Accuracy en el Entrenamiento')
plt.plot(epochs, val_accuracy, 'b', label='Accuracy en la Validación')
plt.title('ACCURACY')
plt.legend()

In [ ]:
plt.plot(epochs, loss, 'ro', label='Pérdida en el entrenamiento')
plt.plot(epochs, val_loss, 'r', label='Pérdida en la validación')
plt.title('LOSS')
plt.legend()

In [ ]:
# predicted_classes = model.predict_classes(X_test)
predicted_classes = np.argmax(model_2_emotion.predict(X_Test), axis=-1)
y_true = np.argmax(y_Test, axis=-1)

In [ ]:
y_true.shape

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, predicted_classes)
plt.figure(figsize = (10, 10))
sns.heatmap(cm, annot = True, cbar = False)

In [ ]:
import json
import keras.backend as K

def deploy(directory, model):
    MODEL_DIR = directory
    version = 1

    # Juntamos el directorio del temp model con la versión elegida
    # El resultado será = '\tmp\version number'
    export_path = os.path.join(MODEL_DIR, str(version))
    print('export_path = {}\n'.format(export_path))

    # Guardemos el modelo con saved_model.save
    # Si el directorio existe, debemos borrarlo con '!rm'
    # rm elimina cada fichero especificado usando la consola de comandos.

    if os.path.isdir(export_path):
        print('\nAlready saved a model, cleaning up\n')
        !rm -r {export_path}

    tf.saved_model.save(model, export_path)

    os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
# Agreguemos el paquete tensorflow-model-server a nuestra lista de paquetes
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
    curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update